In [170]:
# modules
import pandas as pd
import io, requests
from PyPDF2 import PdfFileReader

# establish database
data = pd.DataFrame(columns=["date", "first", "second", "aged", "primary", "state"])
date = pd.to_datetime("2021-08-28")

# begin cycle
while True:
    print(f"checking {date:%b %d, %Y} ...", end="\r")
    url = f"https://www.health.gov.au/sites/default/files/documents/{date:%Y}/{date:%m}/covid-19-vaccine-rollout-update-{date:%-d-%B-%Y}.pdf".lower()
    try:
        request = requests.get(url)
        file = io.BytesIO(request.content)
        reader = PdfFileReader(file)
        if date > pd.to_datetime("2021-07-1"):
            if date > pd.to_datetime("2021-08-15"):
                contents = reader.getPage(5).extractText().split("\n")
            else:
                contents = reader.getPage(2).extractText().split("\n")
            try:
                first = contents[contents.index("dose 1") + 2]
                second = contents[contents.index("dose 2") + 2]
            except:
                first = contents[contents.index("-dose 1") + 2]
                second = contents[contents.index("-dose 2") + 2]        
            data.loc[len(data)] = [date, first, second, None, None, None]            
        else:            
            if date > pd.to_datetime("2021-06-30"):
                contents1 = reader.getPage(5).extractText().split("\n")
                contents2 = reader.getPage(6).extractText().split("\n")                
            else:
                contents1 = reader.getPage(4).extractText().split("\n")                
                contents2 = reader.getPage(5).extractText().split("\n")
            if date > pd.to_datetime("2021-05-17"):
                aged = contents1[contents1.index("Total vaccine doses administered in aged ") + 9]
                primary = contents2[contents2.index("Total vaccine doses administered in ") + 12]
            elif date > pd.to_datetime("2021-05-02"):
                aged = contents1[contents1.index("Total vaccine doses administered in aged ") + 11]
                primary = contents2[contents2.index("Total vaccine doses administered in ") + 13]
            elif date > pd.to_datetime("2021-04-29"):
                aged = contents1[contents1.index("Total vaccine doses administered in aged ") + 9]
                primary = contents2[contents2.index("Total Commonwealth vaccine doses administered in ") + 9]
            elif date > pd.to_datetime("2021-04-26"):
                aged = contents1[contents1.index("Total vaccine doses administered in aged ") + 11]
                primary = contents2[contents2.index("Total vaccine doses administered in ") + 13]
            elif date > pd.to_datetime("2021-04-21"):
                aged = contents1[contents1.index("Total vaccine doses administered in aged ") + 9]
                primary = contents2[contents2.index("Total Commonwealth vaccine doses administered in ") + 9]
            elif date == pd.to_datetime("2021-04-21"):
                aged = "5,403"
                primary = contents2[contents2.index("Total Commonwealth vaccine doses administered in ") + 9]
            elif date == pd.to_datetime("2021-04-19"):
                aged = contents1[contents1.index("Total vaccine doses administered in aged ") + 9]
                primary = "13,248"
            elif date == pd.to_datetime("2021-04-15"):
                aged = "5,366"
                primary = contents2[contents2.index("Total Commonwealth vaccine doses administered in ") + 9]                
            elif date > pd.to_datetime("2021-04-11"):
                aged = contents1[contents1.index("Total vaccine doses administered in aged ") + 9]
                primary = contents2[contents2.index("Total Commonwealth vaccine doses administered in ") + 9]
            elif date == pd.to_datetime("2021-04-11"):
                aged = "5,186"
                primary = "9,186"
            elif date == pd.to_datetime("2021-04-10"):
                aged = "5,186"
                primary = "8,666"
            else:
                aged = contents1[contents1.index("ACT") + 1]
                primary = contents2[contents2.index("ACT") + 1]
            contents3 = reader.getPage(1).extractText().split("\n")
            if date > pd.to_datetime("2021-05-21"):
                state = contents3[contents3.index("Total vaccine doses administered in ") + 20]
            elif date > pd.to_datetime("2021-05-02"):
                state = contents3[contents3.index("Total vaccine doses administered as in ") + 20]
            elif date > pd.to_datetime("2021-04-29"):
                state = contents3[contents3.index("Total vaccine doses administered in ") + 12]
            elif date > pd.to_datetime("2021-04-26"):
                state = contents3[contents3.index("Total vaccine doses administered as in ") + 20]
            else:
                state = contents3[contents3.index("Total vaccine doses administered in ") + 12]
            data.loc[len(data)] = [date, None, None, aged, primary, state]
        date = date - pd.Timedelta(days=1)
    except:
        print()
        print("no data available")
        print(url)
        print(contents1)
        print(contents2)
        print(contents3)
        break

data

checking Apr 11, 2021 ...
no data available
https://www.health.gov.au/sites/default/files/documents/2021/04/covid-19-vaccine-rollout-update-11-april-2021.pdf
['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '95,29246,7751,116529142,067', '', '+3,380', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', '', '', '', '2,883132118', '465,440', '', '+20,216', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '1,166,075', '', '+27,209142,067607,507558,568', '', '+3,380', '+3,613', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']


,date,first,second,aged,primary,state
0,2021-08-28,"219,019","139,112",None,None,None
1,2021-08-27,"217,121","137,456",None,None,None
2,2021-08-26,"213,188","133,495",None,None,None
3,2021-08-25,"210,461","130,437",None,None,None
4,2021-08-24,"208,704","129,260",None,None,None
...,...,...,...,...,...,...
134,2021-04-16,None,None,"5,366","12,259","19,813"
135,2021-04-15,None,None,"5,366","11,693","19,236"
136,2021-04-14,None,None,"5,328","10,996","18,644"
137,2021-04-13,None,None,"5,279","10,419","18,059"


In [172]:
data.to_csv("./vaxData.csv", index=False)

In [171]:
data[["date", "aged", "primary", "state"]].dropna().tail(60)

,date,aged,primary,state
79,2021-06-10,"6,586","52,568","59,586"
80,2021-06-09,"6,555","51,523","58,313"
81,2021-06-08,"6,435","50,583","57,030"
82,2021-06-07,"6,364","49,042","55,787"
83,2021-06-06,"6,364","48,811","54,825"
84,2021-06-05,"6,364","47,560","53,813"
85,2021-06-04,"6,314","46,432","52,596"
86,2021-06-03,"6,256","44,956","51,354"
87,2021-06-02,"6,256","43,913","70,273"
88,2021-06-01,"6,219","42,520","49,508"


In [148]:
request = requests.get("https://www.health.gov.au/sites/default/files/documents/2021/05/covid-19-vaccine-rollout-update-21-may-2021.pdf")
file = io.BytesIO(request.content)
reader = PdfFileReader(file)
contents = reader.getPage(1).extractText().split("\n")
contents

['Total vaccine doses',
 'Commonwealth vaccine doses in ',
 'aged and disability facilities',
 '* Source: Australian Immunisation Register and self reported data.',
 'Numbers are calculated as differences in daily ',
 'cumulative reported numbers to allow for historical corrections in all self',
 '-',
 'reported data sources. ',
 'Past 24 hours',
 'Commonwealth vaccine doses',
 'VIC',
 'QLD',
 'WA',
 'TAS',
 'SA',
 'ACT',
 'NT',
 'NSW',
 'Commonwealth vaccine doses                     ',
 'in primary care',
 '3,472,874',
 'Total vaccine doses ',
 'administered as at ',
 '20 May',
 '+',
 '101,146',
 '320,075',
 '1,933,610',
 '1,219,189',
 'Total vaccine doses administered as in ',
 'State',
 'and Territory',
 'clinics as at ',
 '20 May',
 '(',
 '+',
 '7,001 last 24 hours)',
 '(',
 '+',
 '59,583 last 24 hours)',
 '(',
 '+',
 '34,562 past 24 hours)',
 '326,563',
 '354,942',
 '182,245',
 '142,639',
 '89,108',
 '54,859',
 '41,913',
 '26,920',
 '(+',
 '11,950 ',
 'last',
 '24 hours)',
 '(+',